In [182]:
import os
import pandas as pd
import yfinance as yf
from fredapi import Fred
import datetime

# ==========================================
# КОНФИГУРАЦИЯ
# ==========================================
FRED_API_KEY = "308c46222636bb6d8743d4b4902a42d1"
START_DATE = "2000-08-01" # Updated to user's request
END_DATE = datetime.datetime.now().strftime('%Y-%m-%d')

In [183]:
def collect_enhanced_data():
    print(f"🚀 Запуск финальной сборки датасета (2000 - {datetime.datetime.now().year})...")

    # 1. Yahoo Finance (Рыночные данные)
    yf_tickers = {
        'GC=F': 'Gold_Price',      # Gold price target value
        'SI=F': 'Silver_Price',    # Silver price 
        'DX-Y.NYB': 'DXY_Index',   # индекс доллара США, измеряет силу USD против корзины из 6 валют: 
        '^GSPC': 'SP500',       # Ключевой индекс 500 крупнейших компаний США (80% рыночной капитализации)
        'CL=F': 'WTI_Oil',# фьючерс на WTI Crude Oil (West Texas Intermediate), основной эталон нефти США.
        'HG=F': 'Copper_Price', # Медь
        '^VIX': 'VIX_Index',  #показывает, насколько боятся трейдеры падения рынка в ближайший месяц.
                              # Низкий = все спокойно, высокий = все в панике.
        
        'EURUSD=X': 'USD_EUR',  
        'USDCNY=X': 'USD_CNY', 
        'RUB=X': 'USD_RUB', 
        
        'PL=F': 'Platinum_Price',
        'PA=F': 'Palladium_Price', 
        'GLD': 'GLD_Price',     # ETF на золото, цена отражает спотовую стоимость золота минус 0.4% комиссии.
        'IAU': 'IAU_Price' #  ETF на золото iShares (аналог GLD), содержит физическое золото.
        
    }

    print("📊 Загрузка Yahoo Finance...")
    raw_yf = yf.download(list(yf_tickers.keys()), start=START_DATE, end=END_DATE)

    if isinstance(raw_yf.columns, pd.MultiIndex):
        yf_df = raw_yf['Adj Close'] if 'Adj Close' in raw_yf.columns else raw_yf['Close']
    else:
        yf_df = raw_yf

    yf_df = yf_df.rename(columns=yf_tickers)
    yf_df.index = pd.to_datetime(yf_df.index).tz_localize(None)

    # 2. FRED (Макроэкономика + Опережающие индикаторы)
    fred_series = {
        'FEDFUNDS': 'Fed_Rate', # ключевая ставка ФРС США.
        'DGS10': 'Treasury_10Y', # доходность 10-летних облигаций США.
        'DGS2': 'Treasury_2Y',     # доходность 2-летних облигаций США.
        'DFII10': 'TIPS_10Y',     #  реальная доходность 10-летних облигаций США с защитой от инфляции.
        'CPIAUCSL': 'CPI_Index',  # Индекс потребительских цен США
        'UNRATE': 'Unemployment', # уровень безработицы
        'M2SL': 'M2_Supply', #  сколько денег у населения и компаний на счетах + наличные
        'MANEMP': 'Mfg_Employment', #  для производственной активности
        # 'PMSAUSH066NNBR': 'PMI_Proxy', # показывает скорость роста производства в США.
        
        'DCOILBRENTEU': 'Brent_Oil_Spot',  # спотовая цена нефти Brent (Северное море).
        
        
        'T10YIE' : 'Breakeven_inflation', #ожидаемая инфляция по рынку (разница между обычными Treasuries и TIPS).
        'PPIACO' : 'PPI',#  показывает рост цен, которые платят производители за сырье, энергию и полуфабрикаты
        'A191RL1Q225SBEA': 'GDP_Growth_QoQ', # насколько выросла экономика США за квартал по сравнению с предыдущим (сезонно скорректировано).
    }

    fred_df = pd.DataFrame()
    if FRED_API_KEY:
        try:
            fred = Fred(api_key=FRED_API_KEY)
            print("🏦 Загрузка макроданных FRED...")
            for code, name in fred_series.items():
                try:
                    fred_df[name] = fred.get_series(code, observation_start=START_DATE)
                except:
                    print(f"⚠️ Не удалось загрузить {code}")
        except Exception as e:
            print(f"🔴 Ошибка FRED: {e}")

    print("🧹 Обработка индекса и создание фичей...")
    fred_df.index = pd.to_datetime(fred_df.index).tz_localize(None)

    combined = pd.concat([yf_df, fred_df], axis=1)

    # No resampling to monthly - keeping daily frequency
    df = combined.copy()

    df.index.name = 'Date'
    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date']).dt.date 
    df.set_index('Date', inplace=True)
    



    if 'CPI_Index' in df.columns:
        df['Inflation_MoM'] = df['CPI_Index'].pct_change(12) * 100  # "инфляция за месяц": насколько выросли цены за 12 месяцев.
        
    df['PPI_MoM'] = df['PPI'].pct_change(12) * 100
    
    df['M2_MoM'] = df['M2_Supply'].pct_change(12) * 100

    if 'Treasury_10Y' in df.columns and 'Inflation_MoM' in df.columns:
        df['Real_Rate_Calc'] = df['Treasury_10Y'] - df['Inflation_MoM'] #Простыми словами: сколько настоящих денег вы заработаете после инфляции.

    if 'Treasury_10Y' in df.columns and 'Treasury_2Y' in df.columns:
        df['Yield_Curve_10Y2Y'] = df['Treasury_10Y'] - df['Treasury_2Y'] #  разница между доходностью 10-летних и 2-летних облигаций США

    if 'Gold_Price' in df.columns and 'Silver_Price' in df.columns:
        df['Gold_Silver_Ratio'] = df['Gold_Price'] / df['Silver_Price']
        
    print("🏅 ETF Analysis (оптимизировано)...")

    if 'GLD_Price' in df.columns and 'Gold_Price' in df.columns:
        df['GLD_Return_3M'] = df['GLD_Price'].pct_change(90) # доходность ETF на золото GLD за последние 3 месяца.
        
        df['GLD_Gold_Premium'] = ((df['GLD_Price'] / df['Gold_Price'] - 1) * 100) # насколько цена ETF GLD отклоняется от реальной стоимости золота внутри фонда.
        
        df['Total_ETF_Signal'] = (
            (df['GLD_Return_3M'] > 0.08).astype(int) +      # Приток
            (df['GLD_Gold_Premium'] > -0.5).astype(int) +   # Здоровье
            (df['GLD_Return_3M'] > 0).astype(int)           # Направление
        )
    
    print(f"✅ ETF: Total_ETF_Signal ср. {df['Total_ETF_Signal'].mean():.1f}")

    
    print("📈 CFTC Analysis...")

    df['CFTC_Spec_Bull'] = (
        (df['VIX_Index'] < 25) &                    # Спекулянты активны
        (df['SP500'].pct_change(5) > 0.01) &       # Риск-он
        (df['Gold_Price'].pct_change(10) > 0)      # Momentum
    ).astype(int)       # сколько золота напрокинули крупные спекулянты
    
    print(f"✅ CFTC готово: {df['CFTC_Spec_Bull'].mean():.1%} бычьих дней")
    
    print("⛏️ Mining Cost Index...")

    if all(col in df.columns for col in ['Copper_Price', 'WTI_Oil', 'PPI']):
        df['Mining_Cost_Index'] = (
            0.50 * df['Copper_Price'] +           # Главная составляющая (энергия/оборудование)
            0.30 * df['WTI_Oil'] +                # Дизель для техники  
            0.20 * df['PPI'] / 100                # Инфляция затрат
        ) # средняя себестоимость добычи 1 унции золота для золотодобывающих компаний.
        
        df['Gold_Mining_Margin'] = (df['Gold_Price'] / df['Mining_Cost_Index'] - 1) * 100

        df['Mining_Bull_Signal'] = (df['Gold_Mining_Margin'] > 1.8).astype(int)  # сигнал покупки акций золотодобытчиков
    
    print(f"✅ Mining готово: Margin ср. {df['Gold_Mining_Margin'].mean():.1f}x")

    df['Geopolitics_Shock'] = 0

    # 2008 (Crisis), 2020 (Covid), 2022 (Geopolitics)
    shock_years = [2008, 2009, 2020, 2022, 2023, 2024]
    df.loc[pd.to_datetime(df.index).year.isin(shock_years), 'Geopolitics_Shock'] = 1

    df = df.ffill().dropna(subset=['Gold_Price'])

    output_file = "finolimp_final_daily_dataset_v3.csv"
    df.to_csv(output_file)

    print("\n" + "="*40)
    print("✅ ИДЕАЛЬНЫЙ ДАТАСЕТ СФОРМИРОВАН")
    print(f"📁 Файл: {output_file}")
    print(f"🔑 Индекс: 'Date' (формат datetime)")
    print(f"📈 Новые фичи: Real_Rate_Calc, Yield_Curve_10Y2Y, Geopolitics_Shock")
    print(f"🛠 Обработка Brent: Пропуски заполнены через WTI")
    print("="*40)

if __name__ == "__main__":
    collect_enhanced_data()


🚀 Запуск финальной сборки датасета (2000 - 2026)...
📊 Загрузка Yahoo Finance...


[*********************100%***********************]  14 of 14 completed


🏦 Загрузка макроданных FRED...
🧹 Обработка индекса и создание фичей...
🏅 ETF Analysis (оптимизировано)...
✅ ETF: Total_ETF_Signal ср. 0.8
📈 CFTC Analysis...
✅ CFTC готово: 15.5% бычьих дней
⛏️ Mining Cost Index...
✅ Mining готово: Margin ср. 5803.4x


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26436\2387594698.py:87: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  df['Inflation_MoM'] = df['CPI_Index'].pct_change(12) * 100  # "инфляция за год": насколько выросли цены за 12 месяцев.
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26436\2387594698.py:89: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  df['PPI_MoM'] = df['PPI'].pct_change(12) * 100
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26436\2387594698.py:91: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warni


✅ ИДЕАЛЬНЫЙ ДАТАСЕТ СФОРМИРОВАН
📁 Файл: finolimp_final_daily_dataset_v3.csv
🔑 Индекс: 'Date' (формат datetime)
📈 Новые фичи: Real_Rate_Calc, Yield_Curve_10Y2Y, Geopolitics_Shock
🛠 Обработка Brent: Пропуски заполнены через WTI


In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import datetime

def prepare_pro_data():
    try:
        df = pd.read_csv('finolimp_final_daily_dataset_v3.csv')
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

        df['Gold_Return'] = np.log(df['Gold_Price'] / df['Gold_Price'].shift(1))

        # Лаги доходности (инерция изменений, а не уровней)
        df['Return_Lag1'] = df['Gold_Return'].shift(1)

        # Изменения макро-факторов (First Differences)
        df['d_RealRate'] = df['Real_Rate_Calc'].diff()
        df['d_DXY'] = df['DXY_Index'].diff()

        df = df.dropna()
        return df
    except Exception as e:
        print(f"🔴 Ошибка подготовки: {e}")
        return None
df = prepare_pro_data()
df.columns.shape 

(44,)

In [185]:
df.sample(20)  

,WTI_Oil,DXY_Index,USD_EUR,Gold_Price,GLD_Price,Copper_Price,IAU_Price,Palladium_Price,Platinum_Price,USD_RUB,...,Total_ETF_Signal,CFTC_Spec_Bull,Mining_Cost_Index,Gold_Mining_Margin,Mining_Bull_Signal,Geopolitics_Shock,Gold_Return,Return_Lag1,d_RealRate,d_DXY
Date,,,,,,,,,,,,,,,,,,,,,
2018-06-28,73.449997,95.389999,1.156363,1247.800049,118.220001,2.9535,23.940001,938.299988,850.299988,63.095001,...,0,0,21.697149,5867.604461,0,0,-0.003999,-0.003029,0.000000,0.099998
2021-02-15,59.470001,90.480003,1.212209,1821.599976,170.690002,3.8020,34.720001,2381.300049,1256.400024,73.573196,...,0,0,18.267200,10086.564281,0,0,0.000000,-0.001810,0.000000,0.000000
2015-11-06,44.290001,99.160004,1.088400,1087.599976,104.099998,2.2415,21.000000,612.750000,939.599976,62.250000,...,0,0,14.950750,7352.468391,0,0,-0.015329,-0.001900,0.000000,1.220001
2012-02-09,99.839996,78.559998,1.324854,1739.000000,168.020004,3.9745,33.700001,710.900024,1665.000000,29.725201,...,1,1,31.605450,5427.843979,0,0,0.005594,-0.009840,0.000000,-0.080002
2019-05-28,59.139999,97.959999,1.119382,1276.500000,120.830002,2.7035,24.480000,1344.199951,795.000000,64.340797,...,0,0,20.884900,6035.533583,0,0,-0.005079,0.000000,0.000000,0.349998
2021-02-04,56.230000,91.529999,1.204239,1788.900024,168.149994,3.5660,34.180000,2279.399902,1100.099976,75.885498,...,0,0,18.267200,10086.564281,0,0,-0.023916,0.000928,0.000000,0.409996
2019-03-20,59.830002,95.760002,1.135486,1300.500000,124.180000,2.9155,25.180000,1580.800049,858.200012,64.330200,...,2,0,18.608350,6866.765127,0,0,-0.003454,0.003608,0.000000,-0.619995
2009-01-08,41.700001,81.540001,1.369901,853.900024,84.459999,1.4630,16.888000,191.699997,989.599976,28.358000,...,1,0,15.927550,4763.271458,0,1,0.015104,-0.028481,0.000000,-0.739998
2009-04-13,50.049999,84.639999,1.336398,894.700012,87.879997,2.1255,17.594000,242.350006,1240.400024,33.348999,...,2,0,15.777950,5769.583733,0,1,0.014070,0.000000,0.000000,-1.150002


In [186]:
df.columns

Index(['WTI_Oil', 'DXY_Index', 'USD_EUR', 'Gold_Price', 'GLD_Price',
       'Copper_Price', 'IAU_Price', 'Palladium_Price', 'Platinum_Price',
       'USD_RUB', 'Silver_Price', 'USD_CNY', 'SP500', 'VIX_Index', 'Fed_Rate',
       'Treasury_10Y', 'Treasury_2Y', 'TIPS_10Y', 'CPI_Index', 'Unemployment',
       'M2_Supply', 'Mfg_Employment', 'Brent_Oil_Spot', 'Breakeven_inflation',
       'PPI', 'GDP_Growth_QoQ', 'Inflation_MoM', 'PPI_MoM', 'M2_MoM',
       'Real_Rate_Calc', 'Yield_Curve_10Y2Y', 'Gold_Silver_Ratio',
       'GLD_Return_3M', 'GLD_Gold_Premium', 'Total_ETF_Signal',
       'CFTC_Spec_Bull', 'Mining_Cost_Index', 'Gold_Mining_Margin',
       'Mining_Bull_Signal', 'Geopolitics_Shock', 'Gold_Return', 'Return_Lag1',
       'd_RealRate', 'd_DXY'],
      dtype='object')

In [192]:
new_order = ['DXY_Index', 'd_DXY', 'USD_CNY', 'USD_RUB', 'USD_EUR', 
             'Fed_Rate', 'Treasury_10Y', 'Treasury_2Y', 'TIPS_10Y', 
             'SP500', 'VIX_Index', 'Gold_Silver_Ratio', 
             'WTI_Oil', 'Brent_Oil_Spot', 'Copper_Price', 'Platinum_Price', 'Palladium_Price',
             'CPI_Index', 'Inflation_MoM', 'PPI', 'PPI_MoM', 'M2_Supply',  'M2_MoM',
             # 'PMI_Proxy',
             'Unemployment', 
             'Total_ETF_Signal', 'CFTC_Spec_Bull', 'Mining_Cost_Index', 
             'Gold_Price', 'Silver_Price',   # Все данные которые были в Тз
             # Могут быть лишними
             'GLD_Price', 'IAU_Price', 'Mfg_Employment', 'Breakeven_inflation', 'GDP_Growth_QoQ', 
              'Real_Rate_Calc', 'Yield_Curve_10Y2Y', 'GLD_Return_3M', 
             'GLD_Gold_Premium', 'Gold_Mining_Margin', 'Geopolitics_Shock', 'Gold_Return', 'Return_Lag1',
             'd_RealRate', 'd_DXY']
df1 = df[new_order]

In [188]:
df1 = df1.ffill().dropna(subset=['Gold_Price'])

In [189]:
df1

,DXY_Index,d_DXY,USD_CNY,USD_RUB,USD_EUR,Fed_Rate,Treasury_10Y,Treasury_2Y,TIPS_10Y,SP500,...,Real_Rate_Calc,Yield_Curve_10Y2Y,GLD_Return_3M,GLD_Gold_Premium,Gold_Mining_Margin,Geopolitics_Shock,Gold_Return,Return_Lag1,d_RealRate,d_DXY
Date,,,,,,,,,,,,,,,,,,,,,
2005-03-23,84.000000,0.639999,8.266501,27.690001,1.299292,2.63,4.38,3.59,1.69,1172.530029,...,4.016175,0.79,-0.044164,-90.023519,2516.713918,0,-0.014244,0.000464,0.0,0.639999
2005-03-24,84.129997,0.129997,8.266501,27.667000,1.293193,2.63,4.38,3.59,1.69,1171.420044,...,4.016175,0.79,-0.053372,-90.018837,2516.713918,0,-0.001177,-0.014244,0.0,0.129997
2005-03-25,84.129997,0.000000,8.276500,27.690001,1.295605,2.63,4.38,3.59,1.69,1171.420044,...,4.016175,0.79,-0.056952,-90.018837,2516.713918,0,0.000000,-0.001177,0.0,0.000000
2005-03-28,84.620003,0.490005,8.276400,27.820000,1.289507,2.63,4.38,3.59,1.69,1174.280029,...,4.016175,0.79,-0.049385,-90.011739,2516.713918,0,0.002821,0.000000,0.0,0.490005
2005-03-29,84.320000,-0.300003,8.266501,27.820000,1.293795,2.63,4.38,3.59,1.69,1165.359985,...,4.016175,0.79,-0.055272,-90.009389,2516.713918,0,0.000235,0.002821,0.0,-0.300003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-22,98.360001,-0.400002,6.963700,76.995842,1.167297,3.72,4.09,3.54,1.85,6913.350098,...,3.782645,0.55,0.340743,-90.796324,17912.990660,0,0.015810,0.015055,0.0,-0.400002
2026-01-23,97.599998,-0.760002,6.972600,75.996208,1.175461,3.72,4.09,3.54,1.85,6915.609863,...,3.782645,0.55,0.364639,-90.796190,17912.990660,0,0.013637,0.015810,0.0,-0.760002
2026-01-26,97.040001,-0.559998,6.972600,74.099373,1.185846,3.72,4.09,3.54,1.85,6950.229980,...,3.782645,0.55,0.370069,-90.851822,17912.990660,0,0.020586,0.013637,0.0,-0.559998


In [190]:
df1[['Inflation_MoM', 'PPI_MoM', 'M2_MoM']] = df1[['Inflation_MoM', 'PPI_MoM', 'M2_MoM']].replace(0, np.nan)

df1[['Inflation_MoM', 'PPI_MoM', 'M2_MoM']] = df1[['Inflation_MoM', 'PPI_MoM', 'M2_MoM']].ffill()

df1 = df1.iloc[7:]
df1

,DXY_Index,d_DXY,USD_CNY,USD_RUB,USD_EUR,Fed_Rate,Treasury_10Y,Treasury_2Y,TIPS_10Y,SP500,...,Real_Rate_Calc,Yield_Curve_10Y2Y,GLD_Return_3M,GLD_Gold_Premium,Gold_Mining_Margin,Geopolitics_Shock,Gold_Return,Return_Lag1,d_RealRate,d_DXY
Date,,,,,,,,,,,,,,,,,,,,,
2005-04-01,84.430000,0.370003,8.266501,27.804501,1.290906,2.79,4.46,3.75,1.77,1172.920044,...,4.149280,0.71,-0.061234,-89.992956,2235.330114,0,-0.006553,0.006318,0.133106,0.370003
2005-04-04,84.779999,0.349998,8.266501,27.889999,1.285000,2.79,4.46,3.75,1.77,1176.119995,...,4.149280,0.71,-0.060284,-89.997640,2235.330114,0,-0.004707,-0.006553,0.000000,0.349998
2005-04-05,84.660004,-0.119995,8.266501,27.879999,1.285793,2.79,4.46,3.75,1.77,1181.390015,...,4.149280,0.71,-0.064566,-90.000000,2235.330114,0,0.001414,-0.004707,0.000000,-0.119995
2005-04-06,84.680000,0.019997,8.266501,27.872999,1.286703,2.79,4.46,3.75,1.77,1184.069946,...,4.149280,0.71,-0.051613,-90.018731,2235.330114,0,0.006106,0.001414,0.000000,0.019997
2005-04-07,84.870003,0.190002,8.266501,27.798000,1.285000,2.79,4.46,3.75,1.77,1191.140015,...,4.149280,0.71,-0.066447,-90.018757,2235.330114,0,-0.001406,0.006106,0.000000,0.190002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-22,98.360001,-0.400002,6.963700,76.995842,1.167297,3.72,4.09,3.54,1.85,6913.350098,...,3.782645,0.55,0.340743,-90.796324,17912.990660,0,0.015810,0.015055,0.000000,-0.400002
2026-01-23,97.599998,-0.760002,6.972600,75.996208,1.175461,3.72,4.09,3.54,1.85,6915.609863,...,3.782645,0.55,0.364639,-90.796190,17912.990660,0,0.013637,0.015810,0.000000,-0.760002
2026-01-26,97.040001,-0.559998,6.972600,74.099373,1.185846,3.72,4.09,3.54,1.85,6950.229980,...,3.782645,0.55,0.370069,-90.851822,17912.990660,0,0.020586,0.013637,0.000000,-0.559998


In [191]:
df1.shape

(5502, 44)